In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader, UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# 임베딩 벡터 캐시 저장소소
cache_dir = LocalFileStore("./.cache/")

llm = ChatOpenAI(
    temperature=0.1,
)
loader = UnstructuredFileLoader("./files/1984.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size = 600,
    chunk_overlap = 100,
)

docs = loader.load_and_split(text_splitter = splitter)

embeddings = OpenAIEmbeddings()

# 캐시에 이미 임베딩된 데이터가 있는지 확인 -> 없다면 새로 임베딩 진행
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)


# 문서와 임베딩 모델을 사용해 벡터 저장소에 저장
vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [8]:
# 저장된 벡터들 중 유사한 벡터 찾는 함수
# vectorstore.similarity_search("where does winston live")